Install packages

In [1]:
import sys
!conda install --yes --prefix {sys.prefix} pandas requests lxml geocoder

Solving environment: done

# All requested packages already installed.



Get data from 1 part

In [2]:
import requests
import re
import numpy as np
import pandas as pd

wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page=requests.get(wikipedia_link)
page=raw_wikipedia_page.text

# Get first table from page
df = pd.read_html(page, header =0)[0]
df = df[df['Borough'] != 'Not assigned']
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(", ".join).reset_index()
df['Neighbourhood'] = np.where(df['Neighbourhood'] == 'Not assigned', df['Borough'], df['Neighbourhood'])

/srv/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Get Geospatial Coordinates

In [3]:
geospatial_df = pd.read_csv('Geospatial_Coordinates.csv', header =0)

df = df.merge(geospatial_df, left_on='Postcode', right_on='Postal Code', how='left').drop("Postal Code", axis=1)

Join Geospatial Coordinates to previous data

In [4]:
df.head(12)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [5]:
df.shape

(103, 5)